In [1]:
import pandas as pd
import numpy as np
from sklearn.utils import resample
import math

In [2]:
d_labitems = pd.read_csv('data/mimic-iv-0.4/hosp/d_labitems.csv.gz', compression='gzip')

In [3]:
admissions_sample = pd.read_pickle("admissions_sample.pkl")

## Pull Chart and Lab data only where it matches our sample patient list

In [4]:
# # We have Decided not to use Chart data after exploring it further
# # However, it still comes up in our discussions, so keeping this for now
# # Will need to update this code based on other updates we've made

# %%time #run this for next time

# chunksize = 3*(10 ** 6)
# counter=0
# chartevents_sample = []
# for chunk in pd.read_csv('data/mimic-iv-0.4/icu/chartevents.csv.gz', compression='gzip', chunksize=chunksize):
#     chartevents_sample.append(chunk[chunk['subject_id'].isin(list(admissions_sample.subject_id))])
#     counter+=1
#     print(f'chunk {counter} processed')

# chartevents_final = pd.concat(chartevents_sample)



In [5]:
# chartevents_final.to_pickle("chartevents_final.pkl")

In [6]:
# chartevents_final = pd.read_pickle("chartevents_final.pkl")

In [7]:
# chartevents_final.shape

Load Lab Events table

In [8]:
%%time

chunksize = 3*(10 ** 6)
counter=0
lab_events_sample = []
for chunk in pd.read_csv('data/mimic-iv-0.4/hosp/labevents.csv.gz', compression='gzip', chunksize=chunksize):
    lab_events_sample.append(chunk[chunk['subject_id'].isin(list(admissions_sample.subject_id))])
    counter+=1
    print(f'chunk {counter} processed')

lab_events_final = pd.concat(lab_events_sample)
# 5k patients
# 3min 52s

# 10k patients
# 4min 6s

# 15k patients
# 4min 27s



chunk 1 processed
chunk 2 processed
chunk 3 processed


<string>:2: DtypeWarning: Columns (7,9,12) have mixed types.Specify dtype option on import or set low_memory=False.


chunk 4 processed
chunk 5 processed
chunk 6 processed
chunk 7 processed
chunk 8 processed
chunk 9 processed


<string>:2: DtypeWarning: Columns (9,12) have mixed types.Specify dtype option on import or set low_memory=False.


chunk 10 processed


...

chunk 11 processed
chunk 12 processed
chunk 13 processed
chunk 14 processed
chunk 15 processed


<string>:2: DtypeWarning: Columns (7) have mixed types.Specify dtype option on import or set low_memory=False.


chunk 16 processed
chunk 17 processed
chunk 18 processed
chunk 19 processed
chunk 20 processed
chunk 21 processed
chunk 22 processed
chunk 23 processed


<string>:2: DtypeWarning: Columns (7,9) have mixed types.Specify dtype option on import or set low_memory=False.


chunk 24 processed
chunk 25 processed
chunk 26 processed
chunk 27 processed
chunk 28 processed
chunk 29 processed
chunk 30 processed
chunk 31 processed
chunk 32 processed
chunk 33 processed
chunk 34 processed
chunk 35 processed
chunk 36 processed
chunk 37 processed
chunk 38 processed
chunk 39 processed
chunk 40 processed
chunk 41 processed
CPU times: user 4min 3s, sys: 21.6 s, total: 4min 24s
Wall time: 4min 27s


In [9]:
lab_events_final.to_pickle("lab_events_final.pkl")

In [10]:
# lab_events_final = pd.read_pickle("lab_events_final.pkl")

In [11]:
lab_events_final.shape

(6765210, 15)

## Manipulating the Data

Take only records related to our sample patients

Filter lab events based on first hospital visit of our sample population

In [12]:
# Can we do this filtering when generating lab data?
# For now let's not as we're discussing using different hospitalizations from the same subjects

%time lab_events_sampled = lab_events_final[lab_events_final['hadm_id'].isin(admissions_sample.hadm_id)]


CPU times: user 1.83 s, sys: 707 ms, total: 2.54 s
Wall time: 2.59 s


Check how many patients have had each test done

In [13]:
lab_events_sampled.groupby('itemid')['subject_id'].nunique().sort_values(ascending = False).head(10)

itemid
51221    8433
51265    8297
51222    8236
51301    8235
51248    8229
51279    8229
51277    8229
51250    8229
51249    8229
50912    7985
Name: subject_id, dtype: int64

Most common tests performed on patients with pos diagnosis

In [14]:
lab_events_sampled[lab_events_sampled['subject_id'].isin(list(admissions_sample[admissions_sample['pos'] == 1].subject_id))].groupby('itemid')['subject_id'].nunique().sort_values(ascending = False).head(10)


itemid
50971    1196
50983    1193
50912    1192
50902    1191
51006    1191
51221    1181
51265    1181
50868    1180
50882    1180
50931    1174
Name: subject_id, dtype: int64

Item Black List

In [15]:
item_black_list = [50920]
# 50920 - test results are text in comments - need to come back and figure out how to handle this


Take only the X most commonly performed tests

In [16]:
top_test_num = 40

lab_events_valid = lab_events_sampled[~lab_events_sampled['itemid'].isin(item_black_list)]

# On patients with pos diagnosis
itemid_sub_sample = lab_events_valid[lab_events_valid['subject_id'].isin(list(admissions_sample[admissions_sample['pos'] == 1].subject_id))].groupby('itemid')['subject_id'].nunique().sort_values(ascending = False).head(top_test_num).reset_index().rename(columns = {'index' : 'itemid'}).itemid.to_list()
# On all sample patients
# itemid_sub_sample = lab_events_sampled.groupby('itemid')['subject_id'].nunique().sort_values(ascending = False).head(top_test_num).reset_index().rename(columns = {'index' : 'itemid'}).itemid.to_list()

lab_events_sampled_sub = lab_events_sampled[lab_events_sampled['itemid'].isin(itemid_sub_sample)]


## Data Cleaning

Make a separate table where we run some of the forest models on the Text data?

Special updates for Urine sample tests

In [17]:
# Other invalid value imputation
# For now set them all to normal/most common value
lab_events_sampled_sub.loc[(lab_events_sampled_sub['itemid'] == 51237) & (lab_events_sampled_sub['value'].isin([np.nan, 'ERROR', 'UNABLE TO REPORT'])), ['value', 'valuenum']] = 1.1
lab_events_sampled_sub.loc[(lab_events_sampled_sub['itemid'] == 51237) & (lab_events_sampled_sub['value'].isin(['>21.8'])), ['value', 'valuenum']] = 22
lab_events_sampled_sub.loc[(lab_events_sampled_sub['itemid'] == 51237) & (lab_events_sampled_sub['value'].isin(['>19.2'])), ['value', 'valuenum']] = 20

lab_events_sampled_sub.loc[(lab_events_sampled_sub['itemid'] == 51275) &  (lab_events_sampled_sub['value'].isin(['>150', '>150.0'])), ['value', 'valuenum']] = 175

# Should we just delete these?
# For 5k patients there are 6 records here
lab_events_sampled_sub.loc[(lab_events_sampled_sub['itemid'] == 51274) &  (lab_events_sampled_sub['value'].isin(['ERROR'])), ['value', 'valuenum']] = 11

# Should we just delete these?
# For 5k patients there are 1 records here
lab_events_sampled_sub.loc[(lab_events_sampled_sub['itemid'] == 51275) &  (lab_events_sampled_sub['value'].isin(['UNABLE TO REPORT'])), ['value', 'valuenum']] = 31

# Clumsy..
lab_events_sampled_sub.loc[(lab_events_sampled_sub['itemid'] == 51275) &  (lab_events_sampled_sub['value'].isin(['34..3'])), ['value', 'valuenum']] = 34.3


/opt/anaconda3/lib/python3.7/site-packages/pandas/core/indexing.py:965: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[item] = s


More Data Cleaning

In [32]:
%%time

# Can probably make a helper function combining some of the below mapping

# 51466
lab_events_sampled_sub.loc[(lab_events_sampled_sub['itemid'] == 51466) & (lab_events_sampled_sub['value'].isin([np.nan])), ['value']] = lab_events_sampled_sub['comments']
lab_events_sampled_sub.loc[(lab_events_sampled_sub['itemid'] == 51466) & (lab_events_sampled_sub['value'].isin(['NEG.  ', 'NEG', ' '])), ['value', 'valuenum']] = 0
lab_events_sampled_sub.loc[(lab_events_sampled_sub['itemid'] == 51466) & (lab_events_sampled_sub['value'].isin(['TR', 'TR.  ', 'TR*.  '])), ['value', 'valuenum']] = 1
lab_events_sampled_sub.loc[(lab_events_sampled_sub['itemid'] == 51466) & (lab_events_sampled_sub['value'].isin(['SM', 'SM .  ', 'SM*.  '])), ['value', 'valuenum']] = 2
lab_events_sampled_sub.loc[(lab_events_sampled_sub['itemid'] == 51466) & (lab_events_sampled_sub['value'].isin(['MOD', 'MOD.  ', 'MOD*.  '])), ['value', 'valuenum']] = 3
lab_events_sampled_sub.loc[(lab_events_sampled_sub['itemid'] == 51466) & (lab_events_sampled_sub['value'].isin(['LG', 'LG.  ', 'LG*.  ', 'LGE', 'LRG'])), ['value', 'valuenum']] = 4

lab_events_sampled_sub.loc[(lab_events_sampled_sub['itemid'] == 51466) & (lab_events_sampled_sub['ref_range_lower'].isnull()), 'ref_range_lower'] = 0
lab_events_sampled_sub.loc[(lab_events_sampled_sub['itemid'] == 51466) & (lab_events_sampled_sub['ref_range_upper'].isnull()), 'ref_range_upper'] = 0

# 51514
lab_events_sampled_sub.loc[(lab_events_sampled_sub['itemid'] == 51514) & (lab_events_sampled_sub['value'].isin([np.nan])), ['value']] = lab_events_sampled_sub['comments']
lab_events_sampled_sub.loc[(lab_events_sampled_sub['itemid'] == 51514) & (lab_events_sampled_sub['value'].isin(['NEG.  ', 'NEG', 'NORMAL.  ', ' '])), ['value', 'valuenum']] = 0
lab_events_sampled_sub.loc[(lab_events_sampled_sub['itemid'] == 51514) & (lab_events_sampled_sub['value'].isin(['>8'])), ['value', 'valuenum']] = 10
lab_events_sampled_sub.loc[(lab_events_sampled_sub['itemid'] == 51514) & (lab_events_sampled_sub['value'].isin(['>12.  ', '>12*.  '])), ['value', 'valuenum']] = 15

# 51464
lab_events_sampled_sub.loc[(lab_events_sampled_sub['itemid'] == 51464) & (lab_events_sampled_sub['value'].isin([np.nan])), ['value']] = lab_events_sampled_sub['comments']
lab_events_sampled_sub.loc[(lab_events_sampled_sub['itemid'] == 51464) & (lab_events_sampled_sub['value'].isin(['NEG.  ', 'NEG', ' '])), ['value', 'valuenum']] = 0
lab_events_sampled_sub.loc[(lab_events_sampled_sub['itemid'] == 51464) & (lab_events_sampled_sub['value'].isin(['SM', 'SM .  ', 'SM*.  '])), ['value', 'valuenum']] = 1
lab_events_sampled_sub.loc[(lab_events_sampled_sub['itemid'] == 51464) & (lab_events_sampled_sub['value'].isin(['MOD', 'MOD.  ', 'MOD*.  '])), ['value', 'valuenum']] = 2
lab_events_sampled_sub.loc[(lab_events_sampled_sub['itemid'] == 51464) & (lab_events_sampled_sub['value'].isin(['LG', 'LG.  ', 'LG*.  ', 'LGE'])), ['value', 'valuenum']] = 3

lab_events_sampled_sub.loc[(lab_events_sampled_sub['itemid'] == 51464) & (lab_events_sampled_sub['ref_range_lower'].isnull()), 'ref_range_lower'] = 0
lab_events_sampled_sub.loc[(lab_events_sampled_sub['itemid'] == 51464) & (lab_events_sampled_sub['ref_range_upper'].isnull()), 'ref_range_upper'] = 0

# 51487
# Should Pos be marked abnormal?
lab_events_sampled_sub.loc[(lab_events_sampled_sub['itemid'] == 51487) & (lab_events_sampled_sub['value'].isin([np.nan])), ['value']] = lab_events_sampled_sub['comments']
lab_events_sampled_sub.loc[(lab_events_sampled_sub['itemid'] == 51487) & (lab_events_sampled_sub['value'].isin(['NEG.  ', 'NEG', ' '])), ['value', 'valuenum']] = 0
lab_events_sampled_sub.loc[(lab_events_sampled_sub['itemid'] == 51487) & (lab_events_sampled_sub['value'].isin(['POS.  ', 'POS', 'POS*.  '])), ['value', 'valuenum']] = 1

lab_events_sampled_sub.loc[(lab_events_sampled_sub['itemid'] == 51487) & (lab_events_sampled_sub['ref_range_lower'].isnull()), 'ref_range_lower'] = 0
lab_events_sampled_sub.loc[(lab_events_sampled_sub['itemid'] == 51487) & (lab_events_sampled_sub['ref_range_upper'].isnull()), 'ref_range_upper'] = 0

# 51492
# Should Pos be marked abnormal?
lab_events_sampled_sub.loc[(lab_events_sampled_sub['itemid'] == 51492) & (lab_events_sampled_sub['value'].isin([np.nan])), ['value']] = lab_events_sampled_sub['comments']
lab_events_sampled_sub.loc[(lab_events_sampled_sub['itemid'] == 51492) & (lab_events_sampled_sub['value'].isin(['___'])), ['value', 'valuenum']] = np.nan
lab_events_sampled_sub.loc[(lab_events_sampled_sub['itemid'] == 51492) & (lab_events_sampled_sub['value'].isin(['NEG.  ', 'NEG', ' '])), ['value', 'valuenum']] = 0
lab_events_sampled_sub.loc[(lab_events_sampled_sub['itemid'] == 51492) & (lab_events_sampled_sub['value'].isin(['TR', 'TR.  ', 'TR*.  '])), ['value', 'valuenum']] = 10
lab_events_sampled_sub.loc[(lab_events_sampled_sub['itemid'] == 51492) & (lab_events_sampled_sub['value'].str.contains('TR.')), ['value', 'valuenum']] = 10
lab_events_sampled_sub.loc[(lab_events_sampled_sub['itemid'] == 51492) & (lab_events_sampled_sub['value'].isin(['>300'])), ['value', 'valuenum']] = 350
lab_events_sampled_sub.loc[(lab_events_sampled_sub['itemid'] == 51492) & (lab_events_sampled_sub['value'].isin(['>600.  ', '>600*.  '])), ['value', 'valuenum']] = 700

lab_events_sampled_sub.loc[(lab_events_sampled_sub['itemid'] == 51492) & (lab_events_sampled_sub['ref_range_lower'].isnull()), 'ref_range_lower'] = 0
lab_events_sampled_sub.loc[(lab_events_sampled_sub['itemid'] == 51492) & (lab_events_sampled_sub['ref_range_upper'].isnull()), 'ref_range_upper'] = 0

# 51486
lab_events_sampled_sub.loc[(lab_events_sampled_sub['itemid'] == 51486) & (lab_events_sampled_sub['value'].isin([np.nan])), ['value']] = lab_events_sampled_sub['comments']
lab_events_sampled_sub.loc[(lab_events_sampled_sub['itemid'] == 51486) & (lab_events_sampled_sub['value'].isin(['NEG.  ', 'NEG', ' ', 'N'])), ['value', 'valuenum']] = 0
lab_events_sampled_sub.loc[(lab_events_sampled_sub['itemid'] == 51486) & (lab_events_sampled_sub['value'].isin(['TR', 'TR.  ', 'TR*.  '])), ['value', 'valuenum']] = 1
lab_events_sampled_sub.loc[(lab_events_sampled_sub['itemid'] == 51486) & (lab_events_sampled_sub['value'].isin(['SM', 'SM .  ', 'SM*.  '])), ['value', 'valuenum']] = 2
lab_events_sampled_sub.loc[(lab_events_sampled_sub['itemid'] == 51486) & (lab_events_sampled_sub['value'].isin(['MOD', 'MOD.  ', 'MOD*.  '])), ['value', 'valuenum']] = 3
lab_events_sampled_sub.loc[(lab_events_sampled_sub['itemid'] == 51486) & (lab_events_sampled_sub['value'].isin(['LG', 'LG.  ', 'LG*.  ', 'LGE'])), ['value', 'valuenum']] = 4

lab_events_sampled_sub.loc[(lab_events_sampled_sub['itemid'] == 51486) & (lab_events_sampled_sub['ref_range_lower'].isnull()), 'ref_range_lower'] = 0
lab_events_sampled_sub.loc[(lab_events_sampled_sub['itemid'] == 51486) & (lab_events_sampled_sub['ref_range_upper'].isnull()), 'ref_range_upper'] = 0

# 51506
lab_events_sampled_sub.loc[(lab_events_sampled_sub['itemid'] == 51506) & (lab_events_sampled_sub['value'].isin([np.nan])), ['value']] = lab_events_sampled_sub['comments']
lab_events_sampled_sub.loc[(lab_events_sampled_sub['itemid'] == 51506) & (lab_events_sampled_sub['value'].isin(['Clear.  ', 'CLEAR.  ', 'Clear', ' ', 'CLEAR'])), ['value', 'valuenum']] = 0
lab_events_sampled_sub.loc[(lab_events_sampled_sub['itemid'] == 51506) & (lab_events_sampled_sub['value'].isin(['SlHazy'])), ['value', 'valuenum']] = 0.5
lab_events_sampled_sub.loc[(lab_events_sampled_sub['itemid'] == 51506) & (lab_events_sampled_sub['value'].isin(['Hazy.  ', 'Hazy*.  ', 'Hazy', 'HAZY', 'HAZY*.  ', 'Slcldy', 'SLCLOUDY'])), ['value', 'valuenum']] = 1
lab_events_sampled_sub.loc[(lab_events_sampled_sub['itemid'] == 51506) & (lab_events_sampled_sub['value'].isin(['Cloudy*.  ', 'Cloudy.  ', 'Cloudy', 'CLO', 'CLOUDY', 'CLOU', 'CLOUDY*.  '])), ['value', 'valuenum']] = 2

lab_events_sampled_sub.loc[(lab_events_sampled_sub['itemid'] == 51506) & (lab_events_sampled_sub['ref_range_lower'].isnull()), 'ref_range_lower'] = 0
lab_events_sampled_sub.loc[(lab_events_sampled_sub['itemid'] == 51506) & (lab_events_sampled_sub['ref_range_upper'].isnull()), 'ref_range_upper'] = 0


/opt/anaconda3/lib/python3.7/site-packages/pandas/core/indexing.py:965: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[item] = s


CPU times: user 5.04 s, sys: 132 ms, total: 5.17 s
Wall time: 5.2 s


Additional Data Cleaning

In [33]:
lab_events_sampled_sub.loc[(lab_events_sampled_sub['itemid'] == 51484) & (lab_events_sampled_sub['value'].isin([np.nan, ' ', 'TR', 'NEG'])), ['value', 'valuenum']] = 0
lab_events_sampled_sub.loc[(lab_events_sampled_sub['itemid'] == 51478) & (lab_events_sampled_sub['value'].isin([np.nan, ' ', 'TR', 'NEG'])), ['value', 'valuenum']] = 0

lab_events_sampled_sub.loc[(lab_events_sampled_sub['itemid'] == 51484) & (lab_events_sampled_sub['ref_range_lower'].isnull()), 'ref_range_lower'] = 0
lab_events_sampled_sub.loc[(lab_events_sampled_sub['itemid'] == 51478) & (lab_events_sampled_sub['ref_range_lower'].isnull()), 'ref_range_lower'] = 0

lab_events_sampled_sub.loc[(lab_events_sampled_sub['itemid'] == 51484) & (lab_events_sampled_sub['ref_range_upper'].isnull()), 'ref_range_upper'] = 0
lab_events_sampled_sub.loc[(lab_events_sampled_sub['itemid'] == 51478) & (lab_events_sampled_sub['ref_range_upper'].isnull()), 'ref_range_upper'] = 0

# What are good replacement values for these?
lab_events_sampled_sub.loc[(lab_events_sampled_sub['itemid'] == 51484) & (lab_events_sampled_sub['value'].isin(['>80'])), ['value', 'valuenum']] = 120
lab_events_sampled_sub.loc[(lab_events_sampled_sub['itemid'] == 51478) & (lab_events_sampled_sub['value'].isin(['>1000'])), ['value', 'valuenum']] = 1250

lab_events_sampled_sub.loc[(lab_events_sampled_sub['itemid'] == 51498) & (lab_events_sampled_sub['value'].isin(['<1.005'])), ['value', 'valuenum']] = 1.000
lab_events_sampled_sub.loc[(lab_events_sampled_sub['itemid'] == 51498) & (lab_events_sampled_sub['value'].isin(['<=1.005'])), ['value', 'valuenum']] = 1.000
lab_events_sampled_sub.loc[(lab_events_sampled_sub['itemid'] == 51498) & (lab_events_sampled_sub['value'].isin(['>1.030', '.1.030'])), ['value', 'valuenum']] = 1.035
lab_events_sampled_sub.loc[(lab_events_sampled_sub['itemid'] == 51498) & (lab_events_sampled_sub['value'].isin(['>1.035'])), ['value', 'valuenum']] = 1.040
lab_events_sampled_sub.loc[(lab_events_sampled_sub['itemid'] == 51498) & (lab_events_sampled_sub['value'].isin(['>1.050'])), ['value', 'valuenum']] = 1.055
lab_events_sampled_sub.loc[(lab_events_sampled_sub['itemid'] == 51498) & (lab_events_sampled_sub['comments'].isin(['>1.050*.'])), ['value', 'valuenum']] = 1.055
lab_events_sampled_sub.loc[(lab_events_sampled_sub['itemid'] == 51498) & (lab_events_sampled_sub['value'].isin(['>=1.035'])), ['value', 'valuenum']] = 1.040
lab_events_sampled_sub.loc[(lab_events_sampled_sub['itemid'] == 51498) & (lab_events_sampled_sub['value'].isin([' '])), ['value', 'valuenum']] = 1.015

lab_events_sampled_sub.loc[(lab_events_sampled_sub['itemid'] == 51237) & (lab_events_sampled_sub['value'].isin(['>13.4'])), ['value', 'valuenum']] = 15

lab_events_sampled_sub.loc[(lab_events_sampled_sub['itemid'] == 51491) & (lab_events_sampled_sub['value'].isin([' '])), ['value', 'valuenum']] = 6.0

lab_events_sampled_sub.loc[(lab_events_sampled_sub['itemid'] == 51498) & (lab_events_sampled_sub['value'].isin(['>1.070'])), ['value', 'valuenum']] = 1.080

lab_events_sampled_sub.loc[(lab_events_sampled_sub['itemid'] == 51275) & (lab_events_sampled_sub['value'].isin(['ERROR'])), ['value', 'valuenum']] = np.nan

lab_events_sampled_sub.loc[(lab_events_sampled_sub['itemid'] == 51484) & (lab_events_sampled_sub['value'].astype(float) > 0), 'flag'] = 'abnormal'
lab_events_sampled_sub.loc[(lab_events_sampled_sub['itemid'] == 51478) & (lab_events_sampled_sub['value'].astype(float) > 0), 'flag'] = 'abnormal'

lab_events_sampled_sub.loc[(lab_events_sampled_sub['itemid'] == 51466) & (lab_events_sampled_sub['value'].astype(float) > 0), 'flag'] = 'abnormal'


In [34]:
# Check on differences between value and valuenum columns

lab_events_sampled_sub.loc[((lab_events_sampled_sub['value'].astype(str) + str('.0')) != lab_events_sampled_sub['valuenum'].astype(str)) 
                           & ((lab_events_sampled_sub['value'].astype(str) + str('0')) != lab_events_sampled_sub['valuenum'].astype(str)) 
                           & ((lab_events_sampled_sub['value'].astype(str)) != lab_events_sampled_sub['valuenum'].astype(str)) 
                           & ((lab_events_sampled_sub['value'].astype(str)) != lab_events_sampled_sub['valuenum'].astype(str) + str('0')) 
                           & ((str('0') + lab_events_sampled_sub['value'].astype(str)) != lab_events_sampled_sub['valuenum'].astype(str))
                           & ((lab_events_sampled_sub['value'].astype(str)) != round(lab_events_sampled_sub['valuenum'], 10).astype(str)), ]


,labevent_id,subject_id,hadm_id,specimen_id,itemid,charttime,storetime,value,valuenum,valueuom,ref_range_lower,ref_range_upper,flag,priority,comments
67219781,77231095,16195081,21878501.0,38616318,51498,2119-04-06 01:08:00,2119-04-06 01:14:00,1.000,1.0,,1.001,1.035,abnormal,STAT,NaN


## Code to look into specific test values for cleaning

In [35]:
# lab_events_sampled_sub[lab_events_sampled_sub['value'] == 'N']

In [36]:
# # Used to investigate bad data
# pd.set_option('display.max_rows', 200)

# bad_id = 51486

# # lab_events_sampled_sub[lab_events_sampled_sub['itemid'] == bad_id].value.value_counts().reset_index().sort_values(by = 'index')
# # lab_events_sampled_sub[lab_events_sampled_sub['itemid'] == bad_id][['value', 'comments']].drop_duplicates()
# lab_events_sampled_sub[lab_events_sampled_sub['itemid'] == bad_id][['flag', 'comments', 'value', 'valuenum', 'ref_range_lower', 'ref_range_upper']].drop_duplicates()
# # lab_events_sampled_sub.loc[(lab_events_sampled_sub['itemid'] == bad_id) & (lab_events_sampled_sub['value'].isnull()), ][['value', 'valuenum', 'comments']].drop_duplicates()


In [37]:
# pd.set_option('display.max_rows', 20)

## Get Aggregate Test Values

In [38]:
%time lab_events_sampled_sub_grouped = lab_events_sampled_sub.groupby(['subject_id', 'hadm_id', 'itemid']).agg({'itemid' : ['count'], 'valuenum' : ['min', 'max', 'mean'], 'flag' : ['count'], 'ref_range_lower' : ['min'], 'ref_range_upper' : ['min']})
# 'comments' : ['unique'], 


CPU times: user 413 ms, sys: 59.3 ms, total: 472 ms
Wall time: 477 ms


In [39]:
lab_events_sampled_sub_grouped.reset_index(inplace = True)
lab_events_sampled_sub_grouped.columns = ['subject_id', 'hadm_id', 'itemid', 'count', 'min', 'max', 'mean', 'abn_percent', 'range_min', 'range_max']
# 'comments', 


In [40]:
missing_vals = lab_events_sampled_sub_grouped.isnull().sum(axis = 0) 
missing_vals[missing_vals > 0].sort_values()

min     1925
max     1925
mean    1925
dtype: int64

In [41]:
lab_events_sampled_sub_grouped.loc[lab_events_sampled_sub_grouped['min'].isnull(), ].itemid.value_counts()

# lab_events_sampled_sub_grouped.loc[(lab_events_sampled_sub_grouped['min'].isnull()) & (lab_events_sampled_sub_grouped['itemid'] == 51003), ]#.subject_id.nunique()
# # lab_events_sampled_sub_grouped.subject_id.nunique()

# lab_events_sampled_sub_grouped.loc[lab_events_sampled_sub_grouped['subject_id'] == 10002264, ]


51003    1494
50911     318
51498      51
50885      26
51275      12
50861      11
51274      10
51006       1
51277       1
51265       1
Name: itemid, dtype: int64

In [42]:
lab_events_sampled_sub_grouped['below_min'] = np.where(lab_events_sampled_sub_grouped['min'] < lab_events_sampled_sub_grouped['range_min'], lab_events_sampled_sub_grouped['range_min'] - lab_events_sampled_sub_grouped['min'], 0)
lab_events_sampled_sub_grouped['above_max'] = np.where(lab_events_sampled_sub_grouped['max'] > lab_events_sampled_sub_grouped['range_max'], lab_events_sampled_sub_grouped['max'] - lab_events_sampled_sub_grouped['range_max'], 0)
lab_events_sampled_sub_grouped['abn_percent'] = lab_events_sampled_sub_grouped['abn_percent'] / lab_events_sampled_sub_grouped['count']


## Merge Admissions data so we can group by ethnicity/gender to grab average range min and max

In [43]:
# Here is where we merge Admissions

# lab_events_sampled_sub_grouped.merge(d_icd_diagnoses, left_index = True, left_on = "index", right_on = 'icd_code', how = 'left')


In [44]:
# For now aggregate all patients together

lab_range_min_max = lab_events_sampled_sub_grouped.groupby('itemid').agg({'range_min' : 'mean', 'range_max' : 'mean'})
lab_range_min_max.reset_index(inplace = True)

lab_range_dic = dict(zip(lab_range_min_max['itemid'], zip(lab_range_min_max['range_min'], lab_range_min_max['range_max'])))
lab_range_dic


{50861: (0.0, 40.0),
 50863: (37.88879432624113, 118.05503546099291),
 50868: (8.354329698524696, 19.57280307889673),
 50878: (0.0, 40.0),
 50882: (22.0, 32.0),
 50885: (0.0, 1.5),
 50893: (8.400287894055962, 10.281963437455888),
 50902: (96.0, 108.0),
 50911: (0.0, 10.0),
 50912: (0.45000626174079394, 1.1500062617408338),
 50931: (69.99101296700475, 100.97509307998459),
 50960: (1.599097701959886, 2.5963908078387847),
 50970: (2.70000000000035, 4.5),
 50971: (3.3279234764977663, 5.141885214746697),
 50983: (133.3524015798191, 145.3524015798191),
 51003: (0.0, 0.010000000000000462),
 51006: (6.0, 20.0),
 51221: (37.53089054903356, 49.17787264318748),
 51222: (12.756265177269867, 16.814715881495616),
 51237: (0.9000000000000004, 1.100000000000066),
 51248: (26.642240855510998, 32.0),
 51249: (31.355936322760968, 35.711872645521936),
 51250: (82.0, 98.0),
 51265: (150.0, 425.7779920453176),
 51274: (9.744737334082053, 12.810263600673284),
 51275: (23.964203233256352, 35.98210161662818),


Pivot the table so we have feature columns related to test results

In [45]:
lab_events_sampled_sub_grouped['new_index'] = list(zip(lab_events_sampled_sub_grouped['subject_id'], lab_events_sampled_sub_grouped['hadm_id']))


In [46]:
%time lab_events_sampled_pivot = lab_events_sampled_sub_grouped.pivot(index = 'new_index', columns = 'itemid', values = ['min', 'max', 'mean', 'abn_percent', 'below_min', 'above_max'])
# 'comments', 


CPU times: user 106 ms, sys: 22.7 ms, total: 128 ms
Wall time: 128 ms


In [47]:
lab_events_sampled_pivot = lab_events_sampled_pivot.reset_index()
lab_events_sampled_pivot['subject_id'], lab_events_sampled_pivot['hadm_id'] = zip(*lab_events_sampled_pivot['new_index'])
lab_events_sampled_pivot.drop(['new_index'], axis = 1, inplace = True)


/opt/anaconda3/lib/python3.7/site-packages/pandas/core/generic.py:3936: PerformanceWarning: dropping on a non-lexsorted multi-index without a level parameter may impact performance.
  obj = obj._drop_axis(labels, axis, level=level, errors=errors)


In [48]:
missing_vals = lab_events_sampled_pivot.isnull().sum(axis = 0) 
missing_vals[missing_vals > 0].sort_values()

             itemid
mean         51221      305
min          51221      305
max          51221      305
above_max    51221      305
abn_percent  51221      305
                       ... 
min          50911     6879
mean         50911     6879
             51003     7691
max          51003     7691
min          51003     7691
Length: 240, dtype: int64

## Start Data Imputation

Copy table, retain original for Random Forest

In [49]:
lab_events_impute = lab_events_sampled_pivot.copy()

In [50]:
d_labitems[d_labitems['itemid'].isin(itemid_sub_sample)]

,itemid,label,fluid,category,loinc_code
52,50861,Alanine Aminotransferase (ALT),Blood,Chemistry,1742-6
76,50863,Alkaline Phosphatase,Blood,Chemistry,6768-6
115,50868,Anion Gap,Blood,Chemistry,1863-0
153,50878,Asparate Aminotransferase (AST),Blood,Chemistry,1920-8
212,50882,Bicarbonate,Blood,Chemistry,1963-8
222,51464,Bilirubin,Urine,Hematology,5770-3
229,50885,"Bilirubin, Total",Blood,Chemistry,1975-2
258,51466,Blood,Urine,Hematology,NaN
282,50893,"Calcium, Total",Blood,Chemistry,2000-8
442,50902,Chloride,Blood,Chemistry,2075-0


In [51]:
%%time

np.random.seed(0)

for labitem in lab_range_dic:
    for ind in lab_events_impute[lab_events_impute[('mean', labitem)].isnull()].index:
        val_max = lab_range_dic[labitem][1]
        val_min = lab_range_dic[labitem][0]

        val_ave = (val_max + val_min) / 2
        val_std = (val_max - val_ave) * .333

        ran_vals = np.random.normal(val_ave, val_std, 50)
        impute_min = min(ran_vals)
        impute_max = max(ran_vals)
        impute_mean = np.mean(ran_vals)
        
        val_min = lab_events_impute[('min', labitem)][ind]
        val_max = lab_events_impute[('max', labitem)][ind]
        val_mean = lab_events_impute[('mean', labitem)][ind]
        
        lab_events_impute[('min', labitem)][ind] = np.where(np.isnan(val_min), impute_min, val_min)
        lab_events_impute[('max', labitem)][ind] = np.where(np.isnan(val_max), impute_max, val_max)
        lab_events_impute[('mean', labitem)][ind] = np.where(np.isnan(val_mean), impute_mean, val_mean)
#         patient[('mean', labitem)].replace(np.nan, impute_mean)
#         patient[('abn_count', labitem)].replace(np.nan, 0)
# 5k patients
# 10 items - 2.5s
# 20 items - 6s
# 30 items - 28.6s

# 10k patients
# 40 items - 1m 34s

/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:20: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:21: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:22: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


CPU times: user 1min 4s, sys: 387 ms, total: 1min 4s
Wall time: 1min 5s


In [52]:
for lab_itemid in lab_range_dic:
    lab_events_impute[( 'above_max', lab_itemid)] = lab_events_impute[( 'above_max', lab_itemid)].replace(np.nan, 0)
    lab_events_impute[( 'below_min', lab_itemid)] = lab_events_impute[( 'below_min', lab_itemid)].replace(np.nan, 0)
    lab_events_impute[( 'abn_percent', lab_itemid)] = lab_events_impute[( 'abn_percent', lab_itemid)].replace(np.nan, 0)
#     lab_events_sampled_pivot[( 'comments', lab_itemid)] = lab_events_sampled_pivot[( 'comments', lab_itemid)].replace(np.nan, np.array(np.nan))


In [53]:
missing_vals = lab_events_impute.isnull().sum(axis = 0) 
missing_vals[missing_vals > 0].sort_values()

Series([], dtype: int64)

In [54]:
lab_events_impute

min                                                         \
itemid       50861       50863 50868      50878 50882     50885     50893   
0        36.000000   75.000000  15.0  32.000000  14.0  0.400000  7.700000   
1         2.997088   48.554022   9.0   0.284447  23.0  0.336883  8.699078   
2         9.000000  104.000000  12.0  20.000000  25.0  0.400000  8.500000   
3         8.502958   44.818079  13.0   1.581965  24.0  0.319633  8.700000   
4       108.000000   96.000000  11.0  72.000000  29.0  0.400000  8.800000   
...            ...         ...   ...        ...   ...       ...       ...   
8733      4.598330   34.104847  12.0   5.275434  26.0  0.106361  8.714469   
8734     -1.510000   41.428115  12.0   6.511988  26.0  0.174611  9.100000   
8735      9.000000   64.000000  10.0  11.000000  16.0  0.300000  7.500000   
8736     15.000000   57.000000   5.0  25.000000  22.0  0.800000  7.800000   
8737     28.000000  112.000000   7.0  51.000000  27.0  1.000000  8.100000   

                                ... above_max                                \
itemid  50902      50911 50912  ...     51484 51486 51487 51491 51492 51498   
0        95.0  44.000000   1.1  ...       0.0   4.0   0.0   0.0  30.0   0.0   
1       103.0   1.899695   1.0  ...       0.0   0.0   0.0   0.0   0.0   0.0   
2        91.0   3.000000   1.1  ...       0.0   1.0   0.0   0.0   0.0   0.0   
3       108.0   1.538229   0.6  ...       0.0   0.0   0.0   0.0   0.0   0.0   
4       101.0   1.270150   0.8  ...       0.0   0.0   0.0   0.0   0.0   0.0   
...       ...        ...   ...  ...       ...   ...   ...   ...   ...   ...   
8733    105.0   3.000000   1.1  ...       0.0   0.0   0.0   0.0   0.0   0.0   
8734    104.0   0.318628   1.5  ...       0.0   0.0   0.0   0.0   0.0   0.0   
8735     86.0   1.510547   0.8  ...       0.0   0.0   0.0   0.0   0.0   0.0   
8736     92.0   1.398758   0.5  ...       0.0   4.0   0.0   0.0  30.0   0.0   
8737     90.0   1.885755   0.8  ...       0.0   0.0   0.0   0.0   0.0   0.0   

                   subject_id     hadm_id  
itemid 51506 51514                         
0        1.0   0.0   10002495  24982426.0  
1        0.0   0.0   10002527  29112696.0  
2        0.0   0.0   10005012  28371912.0  
3        0.0   0.0   10010264  26641707.0  
4        0.0   0.0   10010920  24676144.0  
...      ...   ...        ...         ...  
8733     0.0   0.0   19993069  25459716.0  
8734     0.0   0.0   19995478  24108472.0  
8735     0.0   0.0   19996783  25894657.0  
8736     1.0   1.0   19997886  20793010.0  
8737     0.0   1.0   19999204  29046609.0  

[8738 rows x 242 columns]

In [55]:
lab_events_impute_KNN = lab_events_sampled_pivot.copy()

In [56]:
%%time

from sklearn.impute import KNNImputer
imputer = KNNImputer(n_neighbors=math.trunc((lab_events_impute_KNN.shape[0] ** 0.5)))
lab_events_impute_KNN_imputed = imputer.fit_transform(lab_events_impute_KNN)

# 3min 8s

CPU times: user 2min 22s, sys: 32.6 s, total: 2min 55s
Wall time: 2min 50s


In [57]:
lab_events_impute_KNN_imputed = pd.DataFrame(lab_events_impute_KNN_imputed)
lab_events_impute_KNN_imputed.columns = lab_events_impute_KNN.columns
lab_events_impute_KNN_imputed

min                                                         \
itemid       50861       50863 50868      50878 50882     50885     50893   
0        36.000000   75.000000  15.0  32.000000  14.0  0.400000  7.700000   
1        63.720430   97.752688   9.0  73.204301  23.0  1.187097  8.347312   
2         9.000000  104.000000  12.0  20.000000  25.0  0.400000  8.500000   
3        46.021505   97.301075  13.0  52.795699  24.0  0.913978  8.700000   
4       108.000000   96.000000  11.0  72.000000  29.0  0.400000  8.800000   
...            ...         ...   ...        ...   ...       ...       ...   
8733     33.903226   95.591398  12.0  40.161290  26.0  1.144086  8.295699   
8734     37.720430   96.430108  12.0  40.032258  26.0  0.964516  9.100000   
8735      9.000000   64.000000  10.0  11.000000  16.0  0.300000  7.500000   
8736     15.000000   57.000000   5.0  25.000000  22.0  0.800000  7.800000   
8737     28.000000  112.000000   7.0  51.000000  27.0  1.000000  8.100000   

                                ...  above_max                                \
itemid  50902      50911 50912  ...      51484     51486     51487     51491   
0        95.0  44.000000   1.1  ...   0.000000  4.000000  0.000000  0.000000   
1       103.0   5.161290   1.0  ...   8.870968  1.182796  0.086022  0.010753   
2        91.0   3.000000   1.1  ...   0.000000  1.000000  0.000000  0.000000   
3       108.0   5.860215   0.6  ...   3.817204  1.086022  0.075269  0.010753   
4       101.0   7.086022   0.8  ...  10.376344  0.817204  0.021505  0.016129   
...       ...        ...   ...  ...        ...       ...       ...       ...   
8733    105.0   3.000000   1.1  ...  11.612903  0.903226  0.043011  0.000000   
8734    104.0   5.322581   1.5  ...   7.688172  1.172043  0.043011  0.000000   
8735     86.0   6.440860   0.8  ...   0.000000  0.000000  0.000000  0.000000   
8736     92.0   5.623656   0.5  ...   0.000000  4.000000  0.000000  0.000000   
8737     90.0   5.526882   0.8  ...   0.000000  0.000000  0.000000  0.000000   

                                                 subject_id     hadm_id  
itemid      51492     51498     51506     51514                          
0       30.000000  0.000000  1.000000  0.000000  10002495.0  24982426.0  
1       43.440860  0.000495  0.419355  0.612903  10002527.0  29112696.0  
2        0.000000  0.000000  0.000000  0.000000  10005012.0  28371912.0  
3       28.010753  0.000452  0.376344  0.483871  10010264.0  26641707.0  
4       25.161290  0.000538  0.290323  0.666667  10010920.0  24676144.0  
...           ...       ...       ...       ...         ...         ...  
8733    29.032258  0.000430  0.333333  0.677419  19993069.0  25459716.0  
8734    38.978495  0.000140  0.489247  0.483871  19995478.0  24108472.0  
8735     0.000000  0.000000  0.000000  0.000000  19996783.0  25894657.0  
8736    30.000000  0.000000  1.000000  1.000000  19997886.0  20793010.0  
8737     0.000000  0.000000  0.000000  1.000000  19999204.0  29046609.0  

[8738 rows x 242 columns]

In [58]:
for lab_itemid in lab_range_dic:
    lab_events_impute_KNN_imputed[( 'above_max', lab_itemid)] = lab_events_impute_KNN_imputed[( 'above_max', lab_itemid)].replace(np.nan, 0)
    lab_events_impute_KNN_imputed[( 'below_min', lab_itemid)] = lab_events_impute_KNN_imputed[( 'below_min', lab_itemid)].replace(np.nan, 0)
    lab_events_impute_KNN_imputed[( 'abn_percent', lab_itemid)] = lab_events_impute_KNN_imputed[( 'abn_percent', lab_itemid)].replace(np.nan, 0)
#     lab_events_sampled_pivot[( 'comments', lab_itemid)] = lab_events_sampled_pivot[( 'comments', lab_itemid)].replace(np.nan, np.array(np.nan))



## Standardize the Data for use in Models

In [59]:
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()

In [60]:
labs_scaled = lab_events_impute
labs_scaled_KNN = lab_events_impute_KNN_imputed
labs_scaled_missing_data = lab_events_sampled_pivot

In [61]:
scale_cols = ~labs_scaled.columns.isin([('subject_id', ''), ('hadm_id', '')])

scaler.fit(labs_scaled.loc[:, scale_cols])
labs_scaled.loc[:, scale_cols] = scaler.transform(labs_scaled.loc[:, scale_cols])

scaler.fit(labs_scaled_KNN.loc[:, scale_cols])
labs_scaled_KNN.loc[:, scale_cols] = scaler.transform(labs_scaled_KNN.loc[:, scale_cols])

scaler.fit(labs_scaled_missing_data.loc[:, scale_cols])
labs_scaled_missing_data.loc[:, scale_cols] = scaler.transform(labs_scaled_missing_data.loc[:, scale_cols])


## Save Lab Results for Models

In [62]:
# Save non-imputed file as well for forest models

pos_final_test = labs_scaled.merge(admissions_sample, how='left', left_on='hadm_id', right_on='hadm_id')
pos_KNN_final_test = labs_scaled_KNN.merge(admissions_sample, how='left', left_on='hadm_id', right_on='hadm_id')
pos_final_missing_data_test = labs_scaled_missing_data.merge(admissions_sample, how='left', left_on='hadm_id', right_on='hadm_id')


/opt/anaconda3/lib/python3.7/site-packages/pandas/core/reshape/merge.py:618: UserWarning: merging between different levels can give an unintended result (2 levels on the left, 1 on the right)
  warnings.warn(msg, UserWarning)


In [63]:
pos_final_test.to_csv('pos_final_test.csv.gz', compression="gzip")
pos_KNN_final_test.to_csv('pos_KNN_final_test.csv.gz', compression="gzip")
pos_final_missing_data_test.to_csv('pos_final_missing_data_test.csv.gz', compression="gzip")


## Check for Multicollinearity

In [64]:
from statsmodels.stats.outliers_influence import variance_inflation_factor
from statsmodels.tools.tools import add_constant

In [65]:
%%time

labs_scaled_features = labs_scaled.loc[:, ('min', 50868) : ('above_max', 51484)]
add_constant(labs_scaled_features)

high_vif = pd.Series([variance_inflation_factor(labs_scaled_features.values, i) 
               for i in range(labs_scaled_features.shape[1])], 
              index=labs_scaled_features.columns).sort_values(ascending = False)

high_vif.head(20)


/opt/anaconda3/lib/python3.7/site-packages/statsmodels/stats/outliers_influence.py:193: RuntimeWarning: divide by zero encountered in double_scalars
  vif = 1. / (1. - r_squared_i)
/opt/anaconda3/lib/python3.7/site-packages/statsmodels/regression/linear_model.py:1687: RuntimeWarning: invalid value encountered in double_scalars
  return 1 - self.ssr/self.uncentered_tss


CPU times: user 4min 7s, sys: 33.7 s, total: 4min 41s
Wall time: 48.3 s


           itemid
above_max  51484              inf
           51478              inf
max        51466              inf
           51464              inf
           51484              inf
           51478              inf
above_max  51466              inf
           51464              inf
max        51003     1.232585e+06
above_max  51003     1.232257e+06
max        50878     1.433569e+04
above_max  50878     1.426917e+04
max        50861     4.545681e+03
above_max  50861     4.497713e+03
max        51237     1.230869e+03
above_max  51237     1.168241e+03
mean       51248     8.554668e+02
           51279     6.547015e+02
max        50931     6.054340e+02
mean       51222     6.054204e+02
dtype: float64

In [68]:
labs_scaled_features_limited = labs_scaled_features.copy()
labs_scaled_features_limited.drop([('above_max', 51484),
                                  ('above_max', 51478),
                                  ('above_max', 51466),
                                  ('above_max', 51464),
                                  ('max', 51003),
                                  ('max', 50878),
                                  ('max', 50861),
                                  ('max', 51237),
                                  ('mean', 51248),
                                  ('max', 50931),
                                  ('mean', 51222),
                                  ('max', 51274),
                                  ('mean', 51279),
                                  ('mean', 51274),
                                  ('mean', 51478),
                                  ('mean', 50885),
                                  ('mean', 51250),
                                  ('max', 50911),
                                  ('mean', 51003),
                                  ('mean', 51466),
                                  ('mean', 51484),
                                  ('mean', 51486),
                                  ('mean', 51277),
                                  ('mean', 51221),
                                  ('max', 50912),
                                  ('max', 51275),
                                  ('mean', 51492),
                                  ('mean', 51514),
                                  ('mean', 51006),
                                  ('min', 51222),
                                  ('max', 50960)], axis = 1, inplace = True)


In [69]:
%%time

high_vif = pd.Series([variance_inflation_factor(labs_scaled_features_limited.values, i) 
               for i in range(labs_scaled_features_limited.shape[1])], 
              index=labs_scaled_features_limited.columns).sort_values(ascending = False)

high_vif.head(20)


CPU times: user 2min 59s, sys: 25.9 s, total: 3min 24s
Wall time: 35.2 s


           itemid
mean       51249     96.204266
max        51248     87.709068
           50885     87.649260
above_max  50885     85.470891
mean       51506     84.879426
min        51279     70.651643
mean       50912     70.345144
max        51279     70.196776
min        51221     68.809948
below_min  51221     68.510761
min        51248     67.763923
mean       51265     67.695795
           51464     64.278777
min        51250     62.715902
max        51222     60.686303
below_min  51279     58.371683
mean       51487     57.477935
below_min  51222     57.049252
max        51301     57.014977
           51250     50.106411
dtype: float64